<a href="https://colab.research.google.com/github/adadoun/KplerNextDestination/blob/main/lgbmDestinationProbability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download Needed Libraries

In [1]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Libraries & Load Data

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from category_encoders import TargetEncoder
import lightgbm as lgb
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from typing import List, Tuple, Dict, Any
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Load data
print("Loading data...")
train_df = pd.read_csv('drive/MyDrive/Collab_DATA/KplerData/train_trades_csv_prepared.csv')
test_df = pd.read_csv('drive/MyDrive/Collab_DATA/KplerData/test_trades_csv_prepared.csv')

Loading data...


## Train/Val Split: We use a temporal split due to the nature of the data

In [5]:
cat_features = ['vessel_id',
                'origin', 'destination',
                'origin_h3_res2_index', 'destination_h3_res2_index',
                'product_family', 'vessel_type',
                'products',
               'flag_name', 'origin_country_code', 'destination_country_code',
               'previous_visited_port_1', 'previous_visited_port_2', 'previous_visited_port_3',
               'probability_level'
               ]
num_features = ['day_of_week', 'month', 'traded_volume',
               'dead_weight', 'vessel_age', 'origin_draught_change', 'destination_draught_change',
                'origin_cargo_volume', 'destination_cargo_volume', 'od_distance', 'p1_destination_probability',
                'p2_destination_probability', 'p3_destination_probability',
                'p4_destination_probability', 'merged_destination_probability'
               ]

# Split the data
# Sort the dataframe by end_date_time
df_sorted = train_df.sort_values('start_date_time')

# Determine the split point (90% train, 10% validation)
split_index = int(len(df_sorted) * 0.95)

# Split the data
train_df = df_sorted.iloc[:split_index]
val_df = df_sorted.iloc[split_index:]

# Separate features and target
X_train = train_df[cat_features + num_features]  # Replace feature_columns with your actual feature column names
y_train = train_df['is_visit']    # Replace target_column with your actual target column name

X_val = val_df[cat_features + num_features]
y_val = val_df['is_visit']

# Print some information about the split
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Training data date range: {train_df['start_date_time'].min()} to {train_df['start_date_time'].max()}")
print(f"Validation data date range: {val_df['start_date_time'].min()} to {val_df['start_date_time'].max()}")

# If you need to reset index
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

Training set size: 565715
Validation set size: 29775
Training data date range: 2023-01-01T00:35:13.000Z to 2023-10-07T04:39:12.000Z
Validation data date range: 2023-10-07T04:39:12.000Z to 2023-10-30T18:35:58.000Z


## Data Preparation for Model Training

In [6]:
# Preprocess categorical features
print("Preprocessing data...")
high_cardinality_features = ['vessel_id',
                            'origin', 'destination',
                            'origin_h3_res2_index', 'destination_h3_res2_index',
                             'products', 'flag_name', 'origin_country_code', 'destination_country_code',
               'previous_visited_port_1', 'previous_visited_port_2', 'previous_visited_port_3',]
low_cardinality_features = [f for f in cat_features if f not in high_cardinality_features]


Preprocessing data...


In [7]:
# Target encoding for high cardinality features
te = TargetEncoder()
X_train_te = te.fit_transform(X_train[high_cardinality_features], y_train)
X_val_te = te.transform(X_val[high_cardinality_features])


In [8]:
# One-hot encoding for low cardinality features
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_ohe = ohe.fit_transform(X_train[low_cardinality_features])
X_val_ohe = ohe.transform(X_val[low_cardinality_features])


In [9]:
# Combine encoded features with numerical features
X_train_encoded = np.hstack([X_train_te, X_train_ohe, X_train[num_features]])
X_val_encoded = np.hstack([X_val_te, X_val_ohe, X_val[num_features]])

# Prepare LightGBM datasets
train_data = lgb.Dataset(X_train_encoded, label=y_train)
val_data = lgb.Dataset(X_val_encoded, label=y_val, reference=train_data)

## Hyper-parameter Tuning

In [16]:
# Define the parameter grid for randomized search
param_grid = {
    'num_leaves': [31, 63, 127],
    'learning_rate': [0.01, 0.1],
    'feature_fraction': [0.8, 0.9],
    'n_estimators': [100, 150 , 200],
    'max_depth': [-1, 5, 10, 15, 20],
    'min_data_in_leaf': [100, 500, 1000],
}

# Define the base model
base_model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', random_state=42)

# Set up Time Series cross-validation
tscv = TimeSeriesSplit(n_splits=3)

# Custom scoring function that respects time-based splits
def time_based_cv_score(estimator, X, y):
    scores = []
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        estimator.fit(X_train, y_train)
        scores.append(estimator.score(X_test, y_test))
    return np.mean(scores)

# Perform randomized search with time-based cross-validation
print("Performing hyperparameter optimization...")
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_grid,
    n_iter=10,  # number of parameter settings that are sampled
    cv=tscv,
    scoring=time_based_cv_score,
    verbose=1,
    random_state=42,
    n_jobs=-1  # use all available cores
)

# Fit the random search model
random_search.fit(X_train_encoded, y_train)

# Print the best parameters and score
print("Best parameters found:")
print(random_search.best_params_)
print(f"Best cross-validation score: {random_search.best_score_:.4f}")

# Get the best parameters
best_params = random_search.best_params_


Performing hyperparameter optimization...
Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:979: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 110502, number of negative: 455213
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.109591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5521
[LightGBM] [Info] Number of data points in the train set: 565715, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.195332 -> initscore=-1.4157

## Model Training with optimal hyperparameters

In [49]:
# Train the final model with best parameters
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

print("Training final LightGBM model with best parameters...")
# Set callback parameters
callbacks = [
    lgb.early_stopping(stopping_rounds=50, verbose=True),
    lgb.log_evaluation(period=100)
]

# Train the model
print("Training LightGBM model...")
lgbm_model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, val_data],
    callbacks=callbacks
)
# Make predictions on validation set
val_preds = lgbm_model.predict(X_val_encoded)
val_preds_binary = (val_preds > 0.5).astype(int)

# Calculate metrics for validation set
val_accuracy = accuracy_score(y_val, val_preds_binary)
val_auc = roc_auc_score(y_val, val_preds)
val_precision = precision_score(y_val, val_preds_binary)
val_recall = recall_score(y_val, val_preds_binary)
val_f1 = f1_score(y_val, val_preds_binary)

print("Validation metrics:")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"AUC: {val_auc:.4f}")
print(f"Precision: {val_precision:.4f}")
print(f"Recall: {val_recall:.4f}")
print(f"F1 Score: {val_f1:.4f}")

Training final LightGBM model with best parameters...
Training LightGBM model...
[LightGBM] [Info] Number of positive: 110502, number of negative: 455213
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.094900 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5545
[LightGBM] [Info] Number of data points in the train set: 565715, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.195332 -> initscore=-1.415732
[LightGBM] [Info] Start training from score -1.415732
Training until validation scores don't improve for 50 rounds
[100]	training's binary_logloss: 0.203089	valid_1's binary_logloss: 0.170658
[200]	training's binary_logloss: 0.187996	valid_1's binary_logloss: 0.159452
[300]	training's binary_logloss: 0.177915	valid_1's binary_logloss: 0.153024
[400]	training's binary_logloss: 0.165906	valid_1's binary_logloss

## LGBM Model Evaluation

### Score Test Data

In [50]:
# Prepare test data
print("Preparing test data and making predictions...")
X_test = test_df[cat_features + num_features]
X_test_te = te.transform(X_test[high_cardinality_features])
X_test_ohe = ohe.transform(X_test[low_cardinality_features])
X_test_encoded = np.hstack([X_test_te, X_test_ohe, X_test[num_features]])

# Make predictions on test set
test_preds = lgbm_model.predict(X_test_encoded)

Preparing test data and making predictions...


### Build Ranked Destinations list for each vessel

In [51]:
# Create a DataFrame with predictions
ranked_destinations = test_df[['vessel_id', 'destination', 'is_visit', 'vessel_type', 'product_family']].copy()
ranked_destinations['predicted_probability'] = test_preds

# Filter predictions (is_visit = 0) and actual destinations (is_visit = 1)
ranked_destinations_filtered = ranked_destinations[ranked_destinations['is_visit'] == 0]
test_df_sample = ranked_destinations[ranked_destinations['is_visit'] == 1]

# Create a dictionary of actual destinations
actual_destinations = test_df_sample.set_index('vessel_id')['destination'].to_dict()

### Create utilities functions for Model Evaluation

In [52]:
def get_top_n_predictions(group: pd.DataFrame, n: int) -> List[str]:
    """
    Get the top N predictions for a group based on predicted probability.

    Args:
        group (pd.DataFrame): DataFrame containing predictions for a single vessel.
        n (int): Number of top predictions to return.

    Returns:
        List[str]: List of top N predicted destinations.
    """
    # Return the top N destinations with highest predicted probabilities
    return group.nlargest(n, 'predicted_probability')['destination'].tolist()

def calculate_accuracy_and_ranks(n: int) -> Tuple[float, List[float]]:
    """
    Calculate accuracy and get ranks for Mean Reciprocal Rank (MRR) calculation.

    Args:
        n (int): Number of top predictions to consider.

    Returns:
        Tuple[float, List[float]]: Tuple containing accuracy and list of ranks.
    """
    # Get top N predictions for each vessel
    top_n_predictions = ranked_destinations_filtered.groupby('vessel_id').apply(lambda x: get_top_n_predictions(x, n))

    accuracies = []
    ranks = []
    for vessel_id, preds in top_n_predictions.items():
        actual_dest = actual_destinations.get(vessel_id)
        if actual_dest in preds:
            accuracies.append(1)
            ranks.append(1 / (preds.index(actual_dest) + 1))
        else:
            accuracies.append(0)
            ranks.append(0)

    # Return mean accuracy and list of ranks
    return np.mean(accuracies), ranks

def calculate_accuracy_for_top_n(n: int) -> float:
    """
    Calculate accuracy for top N predictions.

    Args:
        n (int): Number of top predictions to consider.

    Returns:
        float: Accuracy of top N predictions.
    """
    # Get top N predictions for each vessel
    top_n_predictions = ranked_destinations_filtered.groupby('vessel_id').apply(lambda x: get_top_n_predictions(x, n))

    # Calculate accuracy
    accuracy = sum(actual_destinations.get(vessel_id) in preds for vessel_id, preds in top_n_predictions.items()) / len(actual_destinations)

    return accuracy

def calculate_mrr() -> float:
    """
    Calculate Mean Reciprocal Rank (MRR) for all predictions.

    Returns:
        float: Mean Reciprocal Rank.
    """
    def get_reciprocal_rank(group: pd.DataFrame) -> float:
        """
        Calculate reciprocal rank for a single vessel's predictions.

        Args:
            group (pd.DataFrame): DataFrame containing predictions for a single vessel.

        Returns:
            float: Reciprocal rank for the vessel.
        """
        vessel_id = group.name
        actual_dest = actual_destinations.get(vessel_id)
        if actual_dest is None:
            return 0

        # Sort predictions by probability in descending order
        sorted_predictions = group.sort_values('predicted_probability', ascending=False).reset_index(drop=True)

        # Find the rank of the actual destination
        rank = sorted_predictions[sorted_predictions['destination'] == actual_dest].index.min()
        if pd.isna(rank):
            return 0

        # Calculate reciprocal rank (add 1 to rank because indexing starts at 0)
        return 1 / (rank + 1)

    # Calculate reciprocal rank for each vessel
    reciprocal_ranks = ranked_destinations_filtered.groupby('vessel_id').apply(get_reciprocal_rank)

    # Print debug information
    print(f"Number of vessels: {len(reciprocal_ranks)}")
    print(f"Number of non-zero ranks: {(reciprocal_ranks > 0).sum()}")
    print(f"Average reciprocal rank: {reciprocal_ranks.mean():.4f}")

    # Return mean reciprocal rank
    return reciprocal_ranks.mean()

### Compute accuracies and MRR metrics

In [53]:
# Calculate Top 1, Top 3, and Top 10 Accuracies
top_1_accuracy, ranks_1 = calculate_accuracy_and_ranks(1)
top_3_accuracy, _ = calculate_accuracy_and_ranks(3)
top_10_accuracy, _ = calculate_accuracy_and_ranks(10)

# Calculate Mean Reciprocal Rank
mrr = calculate_mrr()

# Print the results
print(f"Top 1 Accuracy: {top_1_accuracy:.4f}")
print(f"Top 3 Accuracy: {top_3_accuracy:.4f}")
print(f"Top 10 Accuracy: {top_10_accuracy:.4f}")
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")

Number of vessels: 6246
Number of non-zero ranks: 5038
Average reciprocal rank: 0.5062
Top 1 Accuracy: 0.4075
Top 3 Accuracy: 0.5751
Top 10 Accuracy: 0.6820
Mean Reciprocal Rank (MRR): 0.5062


### Plot Accuracy evolution for different top-k predictions

In [54]:
# Calculate accuracies for top 1 to top 10
accuracies = [calculate_accuracy_for_top_n(i) for i in range(1, 11)]

# 1. Plot accuracy evolution from top 1 to top 10
fig1 = go.Figure(data=go.Scatter(
    x=list(range(1, 11)),
    y=accuracies,
    mode='lines+markers+text',
    text=[f'{acc:.2%}' for acc in accuracies],
    textposition='top center'
))
fig1.update_layout(
    title='Accuracy Evolution: Top 1 to Top 10',
    xaxis_title='Top N Predictions',
    yaxis_title='Accuracy',
    yaxis_tickformat='.0%',
    xaxis=dict(tickmode='linear', tick0=1, dtick=1)
)
fig1.show()

In [55]:
def calculate_top_1_accuracy(group: pd.DataFrame) -> bool:
    """
    Calculate the top 1 accuracy for a group of predictions.

    Args:
        group (pd.DataFrame): A group of predictions for a single vessel.

    Returns:
        bool: True if the top predicted destination matches the actual destination, False otherwise.
    """
    return actual_destinations.get(group.name) == group.nlargest(1, 'predicted_probability')['destination'].iloc[0]

# Count the number of potential predictions per vessel
ranked_destinations['num_potential_predictions'] = ranked_destinations.groupby('vessel_id')['vessel_id'].transform('count')

# Calculate top 1 accuracy for each vessel
ranked_destinations['top_1_accuracy'] = ranked_destinations.groupby('vessel_id').apply(calculate_top_1_accuracy)

# Filter to include only up to 1000 potential predictions
ranked_destinations = ranked_destinations[ranked_destinations['num_potential_predictions'] <= 1000]

# Define bin parameters
bin_size = 100
max_samples = 1000
bins = range(0, max_samples + bin_size, bin_size)

# Group accuracy data into bins and calculate mean accuracy and count for each bin
accuracy_by_bin = ranked_destinations.groupby(pd.cut(ranked_destinations['num_potential_predictions'], bins=bins))['top_1_accuracy'].agg(['mean', 'count']).reset_index()
accuracy_by_bin['bin_midpoint'] = accuracy_by_bin['num_potential_predictions'].apply(lambda x: x.mid)

# Create subplots: line plot and histogram
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("Line Plot: Top 1 Accuracy vs Number of Potential Predictions (up to 1000)",
                                    "Histogram: Distribution of Potential Predictions (up to 1000)"),
                    vertical_spacing=0.1)

# Add line plot for accuracy
fig.add_trace(
    go.Scatter(x=accuracy_by_bin['bin_midpoint'], y=accuracy_by_bin['mean'], mode='lines+markers',
               name='Top 1 Accuracy', text=[f'{acc:.2%}' for acc in accuracy_by_bin['mean']],
               hovertemplate='Potential Predictions: %{x}<br>Accuracy: %{text}<br>Count: %{customdata}',
               customdata=accuracy_by_bin['count']),
    row=1, col=1
)

# Add histogram for distribution of potential predictions
fig.add_trace(
    go.Bar(x=accuracy_by_bin['bin_midpoint'], y=accuracy_by_bin['count'],
           name='Sample Count', hovertemplate='Potential Predictions: %{x}<br>Count: %{y}'),
    row=2, col=1
)

# Update layout
fig.update_layout(height=800, title_text="Top 1 Accuracy vs Number of Potential Predictions (up to 1000)")
fig.update_xaxes(title_text="Number of Potential Predictions", row=1, col=1, tickmode='array', tickvals=list(range(0, 1001, 100)))
fig.update_xaxes(title_text="Number of Potential Predictions", row=2, col=1, tickmode='array', tickvals=list(range(0, 1001, 100)))
fig.update_yaxes(title_text="Top 1 Accuracy", tickformat='.0%', row=1, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)

# Display the figure
fig.show()

# Print statistics
print(f"Total number of vessels (up to 1000 potential predictions): {len(ranked_destinations['vessel_id'].unique())}")
print(f"Average number of potential predictions per vessel: {ranked_destinations['num_potential_predictions'].mean():.2f}")
print(f"Median number of potential predictions per vessel: {ranked_destinations['num_potential_predictions'].median():.2f}")
print(f"Range of potential predictions: {ranked_destinations['num_potential_predictions'].min()} to {ranked_destinations['num_potential_predictions'].max()}")
print(f"Overall Top 1 Accuracy: {ranked_destinations['top_1_accuracy'].mean():.2%}")
print(f"Correlation between number of potential predictions and accuracy: {accuracy_by_bin['bin_midpoint'].corr(accuracy_by_bin['mean']):.4f}")

# Additional statistics for vessels with more than 1000 potential predictions
vessels_over_1000 = ranked_destinations[ranked_destinations['num_potential_predictions'] > 1000]
print(f"\nNumber of vessels with more than 1000 potential predictions: {len(vessels_over_1000['vessel_id'].unique())}")
if not vessels_over_1000.empty:
    print(f"Average accuracy for vessels with >1000 potential predictions: {vessels_over_1000['top_1_accuracy'].mean():.2%}")


Total number of vessels (up to 1000 potential predictions): 6299
Average number of potential predictions per vessel: 446.24
Median number of potential predictions per vessel: 449.00
Range of potential predictions: 1 to 992
Overall Top 1 Accuracy: 51.68%
Correlation between number of potential predictions and accuracy: 0.2751

Number of vessels with more than 1000 potential predictions: 0


In [56]:
def compute_top_1_accuracy_by_dimension(dimension: str, top_n: int) -> pd.Series:
    """
    Compute top 1 accuracy for different categories within a specified dimension.

    Args:
        dimension (str): The column name of the dimension to analyze.
        top_n (int): The number of top categories to consider.

    Returns:
        pd.Series: A series containing the top 1 accuracy for each category.
    """
    # Get the top N categories for the specified dimension
    top_categories = ranked_destinations[dimension].value_counts().nlargest(top_n).index

    accuracies = {}
    for category in top_categories:
        # Filter the dataset for the current category
        subset = ranked_destinations[ranked_destinations[dimension] == category]
        # Compute and store the mean accuracy for this category
        accuracies[category] = subset.groupby('vessel_id').apply(calculate_top_1_accuracy).mean()

    return pd.Series(accuracies)

# Compute top 1 accuracy for top 4 vessel types
vessel_type_accuracy = compute_top_1_accuracy_by_dimension('vessel_type', 4)

# Compute top 1 accuracy for top 5 product families
product_family_accuracy = compute_top_1_accuracy_by_dimension('product_family', 5)

# Create a subplot figure with two rows
fig3 = make_subplots(rows=2, cols=1, subplot_titles=('Top 1 Accuracy by Vessel Type', 'Top 1 Accuracy by Product Family'))

# Add bar chart for vessel type accuracy
fig3.add_trace(go.Bar(
    x=vessel_type_accuracy.index,
    y=vessel_type_accuracy.values,
    text=[f'{acc:.2%}' for acc in vessel_type_accuracy.values],
    textposition='auto'
), row=1, col=1)

# Add bar chart for product family accuracy
fig3.add_trace(go.Bar(
    x=product_family_accuracy.index,
    y=product_family_accuracy.values,
    text=[f'{acc:.2%}' for acc in product_family_accuracy.values],
    textposition='auto'
), row=2, col=1)

# Update layout of the figure
fig3.update_layout(height=800, title_text="Top 1 Accuracy by Vessel Type and Product Family")

# Update x-axis labels
fig3.update_xaxes(title_text="Vessel Type", row=1, col=1)
fig3.update_xaxes(title_text="Product Family", row=2, col=1)

# Update y-axis labels and format
fig3.update_yaxes(title_text="Top 1 Accuracy", tickformat='.0%', row=1, col=1)
fig3.update_yaxes(title_text="Top 1 Accuracy", tickformat='.0%', row=2, col=1)

# Display the figure
fig3.show()
